In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc, dash_table, Input, Output
import dash_bootstrap_components as dbc

In [47]:
df = pd.read_csv("table.csv")
df

,name,size,Positives,%Positive,MaxSizeTest,F1,AUC
0,1043_ada_agnostic,218976,3430,0.02,1132,0.90,0.76
1,1048_jEdit_4,3321,204,0.06,165,0.76,0.73
2,1049_pc4,55404,1280,0.02,178,0.92,0.59
3,1050_pc3,59394,1403,0.02,160,0.95,0.58
4,1054_mc2,6440,109,0.02,52,0.86,0.81
...,...,...,...,...,...,...,...
156,namao,4101335,24621,0.01,9844,0.98,0.96
157,occupancy,123360,15810,0.13,4750,1.00,0.99
158,phoneme,32424,1586,0.05,3818,0.84,0.89
159,spambase,266858,2788,0.01,1813,0.96,0.95


In [48]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

In [56]:
mid_size = (df['size'].min()+df['size'].max())/2
mid_alpha = (df['%Positive'].min()+df['%Positive'].max())/2
mid_auc = (df['AUC'].min()+df['AUC'].max())/2

app.layout = dbc.Container([
    dcc.Markdown("Datasets for experiments", style={"textAlign":"center"}),
    
    dbc.Label("Show number of rows"),
    
    row_drop := dcc.Dropdown(value=10, clearable=False, style={"width":"35%", 'color':'black'}, 
                            options=[10, 25, 50, 100, 150, 160], id='row_drop'),
    
    my_table := dash_table.DataTable(data=df.to_dict('records'),
                                     id='my_table',
                                    columns=[{"name": i, "id": i} for i in df.columns],
                                    page_size=10,
                                    style_table={'padding':'10px'},

                                    style_header={'backgroundColor': 'rgb(30, 30, 30)','fontWeight': 'bold'},
                                    style_data={'height':'50px','margin':'10px', 'width':'150px', 'minWidth':'150px', 'maxWidth':'150px',
                                               'overflow':'hidden', 'textOverflow':'ellipsis', 
                                               'color': 'white',
                                               'backgroundColor': 'rgb(50, 50, 50)'}),
    
    html.Div([
         dbc.Row([
        dbc.Col(html.Div("Sizes"),style={'text-align':'center','padding-bottom-top':'10px'},width=3),
        dbc.Col(html.Div("Alphas"),style={'text-align':'center','padding-bottom-top':'10px'}, width=3),
        dbc.Col(html.Div("Complexity (AUC)"),style={'text-align':'center','padding-bottom-top':'10px'}, width=3),
    ], justify='between', className='mt-3-mb-4'),
    
    dbc.Row([
        dbc.Col([
            size_range := dcc.RangeSlider(df['size'].min(), 
                                          df['size'].max(), 
                                          500,
                                          value=[df['size'].min(), df['size'].max()], 
                                          marks={
                                    df['size'].min():{'label':df['size'].min(), 'style':{'color':'white'}},
                                    mid_size:{'label':mid_size, 'style':{'color':'white'}},
                                    df['size'].max():{'label':df['size'].max(), 'style':{'color':'white'}
                                           }},
                                          tooltip={"placement": "bottom", "always_visible": True},
                                          id='size_range') 
        ], width=3),
        
        dbc.Col([
            alpha_range := dcc.RangeSlider(df['%Positive'].min(), 
                                           df['%Positive'].max(), 
                                           0.01, 
                                           value=[df['%Positive'].min(), df['%Positive'].max()],
                                           marks={
                        df['%Positive'].min():{'label':df['%Positive'].min(), 'style':{'color':'white'}},
                        mid_alpha:{'label':mid_alpha, 'style':{'color':'white'}},
                        df['%Positive'].max():{'label':df['%Positive'].max(), 'style':{'color':'white'}
                                           }},
                                           tooltip={"placement": "bottom", "always_visible": True},
                                           id='alpha_range') 
        ], width=3),
        
        dbc.Col([
            complex_range := dcc.RangeSlider(df['AUC'].min(), df['AUC'].max(), 0.01,
                                          value=[df['AUC'].min(), df['AUC'].max()],
                                          marks={
                                    df['AUC'].min():{'label':df['AUC'].min(), 'style':{'color':'white'}},
                                    mid_auc:{'label':mid_auc, 'style':{'color':'white'}},
                                    df['AUC'].max():{'label':df['AUC'].max(), 'style':{'color':'white'}}
                                           },
                                          tooltip={"placement": "bottom", "always_visible": True},
                                          id='complex_range') 
        ], width=3),
    ], justify="between", className='mt-3-mb-4')
    ], style={'display':'flex', 'flex-direction':'column'})
], fluid=True)

@callback(
    Output(component_id='my_table', component_property='data'),
    Output(component_id='my_table', component_property='page_size'),
    Input(component_id='row_drop', component_property='value'),
    Input(component_id='size_range', component_property='value'),
    Input(component_id='alpha_range', component_property='value'),
    Input(component_id='complex_range', component_property='value')
)
def update_output(drop_s, size_r, alpha_r, complex_r):
    dff = df.copy()

    if (drop_size):
        data = dff.to_dict('records')

    return data, drop_s
    

app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [54]:
if __name__ == '__main__':
    app.run_server()